In [ ]:
from src.strategies.bond_arb_strategy import BayesianBondArbStrategy
from src.utils.data_loader import load_processed_data
from risk_metrics import StrategyRiskAnalyzer

trades = load_processed_data('backtest_results/latest_trades.pkl')  
spread_series = load_processed_data('bond_spreads_2023')
analyzer = StrategyRiskAnalyzer(trades, initial_capital=1e6)
risk_report = analyzer.generate_risk_report()

# Generate interactive risk dashboard
display(pd.DataFrame([risk_report]).T.rename(columns={0: 'Value'}))

In [ ]:
# VaR method comparison
methods = ['historical', 'gaussian', 'cornish-fisher']
var_results = {method: analyzer.compute_strategy_var(method) for method in methods}
pd.Series(var_results).plot(kind='bar', title='95% VaR Comparison')

In [ ]:
# Drawdown statistics
drawdown_stats = analyzer.compute_trade_drawdowns()
print(f"Max Drawdown: {drawdown_stats['max_drawdown']:.2%}")
print(f"Avg Drawdown Duration: {drawdown_stats['drawdown_duration']:.1f} periods")

In [ ]:
# Stress testing
def stress_test_scenario(spread_shock=0.05):
    shocked_spread = spread_series * (1 + spread_shock)
    shocked_trades = BayesianBondArbStrategy().backtest_strategy(shocked_spread)
    return StrategyRiskAnalyzer(shocked_trades).generate_risk_report()

stress_test_results = pd.DataFrame({
    '5% Widening': stress_test_scenario(0.05),
    '10% Widening': stress_test_scenario(0.10),
    '5% Narrowing': stress_test_scenario(-0.05)
})
stress_test_results.T.plot(subplots=True, layout=(3,2), figsize=(12,8))

In [ ]:
# Liquidity-adjusted VaR
def liquidity_adjusted_var(analyzer, haircut=0.03):
    base_var = analyzer.compute_strategy_var()
    return base_var - haircut * analyzer.compute_trade_drawdowns()['max_drawdown']

print(f"Liquidity-Adjusted VaR: {liquidity_adjusted_var(analyzer):.2%}")

In [ ]:
# Compare different market regimes
regimes = {
    'Low Volatility': '2022-01_2022-04',
    'High Volatility': '2023-03_2023-05'
}

for regime, period in regimes.items():
    period_trades = load_processed_data(f'backtests/{period}_trades')
    analyzer = StrategyRiskAnalyzer(period_trades)
    print(f"\n{regime} Regime:")
    display(analyzer.generate_risk_report())